In [1]:
#우선 파라미터 리스트를 불러오자

In [2]:
import pandas as pd
BestParam = pd.read_csv("BestParam.csv")

In [3]:
import numpy
import pandas
import sys
from sklearn.decomposition import PCA as sklearnPCA
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit import Chem, DataStructs
import matplotlib.pyplot as plt
from scipy.spatial import ConvexHull

In [4]:
import rdkit
import numpy
import pandas
import numpy as np
import pandas as pd

from rdkit import Chem
from rdkit.Chem import AllChem
from rdkit import DataStructs
from rdkit.Chem import Draw
from rdkit.Chem.Draw import SimilarityMaps
from rdkit import Chem, DataStructs
from rdkit.Chem import MACCSkeys

from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import roc_auc_score
from sklearn.metrics import precision_recall_curve
from sklearn.metrics import auc
from sklearn.metrics import confusion_matrix
from imblearn.combine import SMOTEENN 
from pprint import pprint
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import StratifiedKFold
from sklearn.model_selection import train_test_split
from imblearn.pipeline import Pipeline 

In [5]:
selected_assay_list = list(BestParam['Assays'])

In [6]:
selected_assay_list

['TOX21_VDR_BLA_agonist_ratio',
 'TOX21_VDR_BLA_antagonist_ratio',
 'TOX21_HSE_BLA_agonist_ratio',
 'TOX21_ESRE_BLA_ratio',
 'TOX21_FXR_BLA_agonist_ratio',
 'TOX21_FXR_BLA_antagonist_ratio',
 'TOX21_ARE_BLA_agonist_ratio',
 'TOX21_PPARd_BLA_agonist_ratio',
 'TOX21_PPARd_BLA_antagonist_ratio',
 'TOX21_PPARg_BLA_antagonist_ratio',
 'TOX21_ELG1_LUC_Agonist',
 'TOX21_AhR_LUC_Agonist',
 'TOX21_Aromatase_Inhibition',
 'TOX21_AR_BLA_Agonist_ratio',
 'TOX21_AR_BLA_Antagonist_ratio',
 'TOX21_AR_LUC_MDAKB2_Agonist',
 'TOX21_p53_BLA_p1_ratio',
 'TOX21_p53_BLA_p2_ratio',
 'TOX21_p53_BLA_p3_ratio',
 'TOX21_p53_BLA_p4_ratio',
 'TOX21_p53_BLA_p5_ratio',
 'TOX21_ERa_BLA_Agonist_ratio',
 'TOX21_ERa_BLA_Antagonist_ratio',
 'TOX21_GR_BLA_Agonist_ratio',
 'TOX21_GR_BLA_Antagonist_ratio',
 'TOX21_TR_LUC_GH3_Agonist',
 'TOX21_TR_LUC_GH3_Antagonist',
 'TOX21_PPARg_BLA_Agonist_ratio',
 'NVS_ADME_hCYP2B6',
 'NVS_ADME_hCYP1A2',
 'NCCT_TPO_AUR_dn']

In [7]:
assay_name = '335_all_assay'
df = pd.read_csv('./Data/'+assay_name+"_df.csv")

In [8]:
save_path = './31assays_AD/'
!mkdir {save_path}
!mkdir {save_path+"dup_smiles/"}
!mkdir {save_path+"err_smiles/"}
!mkdir {save_path+"performance/"}
!mkdir {save_path+"params/"}

mkdir: cannot create directory ‘./31assays_AD/’: File exists
mkdir: cannot create directory ‘./31assays_AD/dup_smiles/’: File exists
mkdir: cannot create directory ‘./31assays_AD/err_smiles/’: File exists
mkdir: cannot create directory ‘./31assays_AD/performance/’: File exists
mkdir: cannot create directory ‘./31assays_AD/params/’: File exists


In [21]:
def g_tf(assay_index) :
    ss = selected_assay_list[assay_index] # 그룹
    ffpp = "maccs"

    df_ss = df.loc[:,['smiles',ss]]
    df_ss = df_ss.dropna()
    assays_ss = df_ss.columns
    sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

        #Smiles --> MACCS Keys
    assay = ss
    df_train  = df_ss[['smiles',ss] ]

    smiles_list = []
    err_smiles = []# fingerfrint로 변환시 에러 smiles
    fps = []
    enc_y = []
    for index, row in df_train.iterrows():

      if row[-1] == 'N' : 
        target = 0
      else :
        target = 1

      try : 
        mol = Chem.MolFromSmiles(row['smiles'])

        if ffpp == 'maccs' :    
            fp = MACCSkeys.GenMACCSKeys(mol)
        elif ffpp == 'morgan' : 
            fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2)
        elif ffpp == 'rdkit' : 
            fp = Chem.RDKFingerprint(mol)
        elif ffpp == 'pattern' : 
            fp = Chem.rdmolops.PatternFingerprint(mol)
        elif ffpp == 'layerd' : 
            fp = Chem.rdmolops.LayeredFingerprint(mol)
        else : 
            print("fingerfrint selection error!")

        smiles_list.append(row['smiles'])   
        fps.append(fp)
        enc_y.append(target)
      except : 
        err_smiles.append(row['smiles'])
        pass

    np_fps = []
    for fp in fps:
      arr = numpy.zeros((1,))
      DataStructs.ConvertToNumpyArray(fp, arr)
      np_fps.append(arr)

    np_fps_array = numpy.array(np_fps)

    print("변환 에러난 smiles 개수 : ", len(err_smiles), err_smiles)

    x_train_res, y_train_res = np_fps_array, np.array(enc_y)
    train = pd.DataFrame(data=x_train_res)
    train['label'] = y_train_res

    d = pd.DataFrame(columns = ['smiles','Type'] )
    d['smiles'] = smiles_list
    d['Type'] = enc_y

    #모든 물질에 대한 fingerprint를 string형태로 바꾼다(비교하기위함).
    mychems = []
    for item in np_fps:
        #print(item)
        item = np.array(item,dtype=int) #왜 굳이 두번? int형으로 추가하려면 이렇게 하면 되는구나....
        s = item.tolist() #넘파이어레이를 리스트로 바꾼다. 
        t = [str(i) for i in s]
        mychems.append(''.join(t))

    i = 0
    counter = 0
    chemdict = {} #딗셔너리.
    listofdups = []
    for item in mychems:
        try:
            chemdict[str(item)].append(i)
            #print('dup found')
            for j in range(len(chemdict[str(item)])):
                if (d['Type'][i] != d['Type'][chemdict[str(item)][j]]) :
                    listofdups.append(i)
                    counter+=1
        except: 
            #print('adding new')
            chemdict[str(item)] = []
            chemdict[str(item)].append(i)
        i+=1
    print(str(counter)+' '+str(len(list(set(listofdups)))))
    #smote적용하기 전에서 listofdups를 빼면 된다!

    dups_list = list(set(listofdups))
    pprint(dups_list) # 0인것만 지우는게 맞나?

    dups_list_0 = []
    for index in dups_list : 
      if train.iloc[index].label == 0 :
        dups_list_0.append(index)

    train = train.drop(dups_list_0)

    duplicated_smiles_0_df = pd.DataFrame()
    duplicated_smiles_0_df['deleted_duplicate_smiles_0'] = smiles_list
    duplicated_smiles_0_df = duplicated_smiles_0_df.iloc[dups_list_0,]
    duplicated_smiles_0_df

    save_path = './31assays_AD/'
    dupdf = save_path+"dup_smiles/"+ss+"_dup_smiles_df.csv"
    duplicated_smiles_0_df.to_csv(dupdf)

    err_smiles_df = pd.DataFrame(columns = [ss], index = ['err_n','err_smiles'])
    err_smiles_df[ss] = [ len(err_smiles), err_smiles]

    ersdf = save_path+"err_smiles/"+ss+"_err_smiles_df.csv"
    err_smiles_df.to_csv(ersdf)


    skf = StratifiedKFold(n_splits=10)
    sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

    X = np.array( train.drop('label', axis =1) )
    y = np.array( train.label )

    #train test split
    X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y,
                                                        stratify=y, 
                                                        test_size=0.1,
                                                        random_state = 42)


    sklearn_pca = sklearnPCA(n_components=2)
    y_sklearn = sklearn_pca.fit_transform(X_train_valid)
    pccr = pandas.DataFrame(data=y_sklearn)
    pccr.columns = ['PC1', 'PC2']

    y_sklearn2 = sklearn_pca.fit_transform(X_test)
    pccr2 = pandas.DataFrame(data=y_sklearn2)
    pccr2.columns = ['PC1', 'PC2']

    def is_p_inside_points_hull(points, p):
        """Check if a point is inside the convex hull."""
        hull = ConvexHull(points)
        new_points = numpy.append(points, p, axis=0)
        new_hull = ConvexHull(new_points)
        if list(hull.vertices) == list(new_hull.vertices):
            return True
        else:
            return False

    inside_AD = []
    outside_AD = []
    for i in range(0, len(pccr2)):
        if is_p_inside_points_hull(pccr, pccr2.loc[[i]]) : 
            inside_AD.append(i)
        else : 
            outside_AD.append(i)

    print('inside_AD : ', len(inside_AD))
    print('outside_AD : ', len(outside_AD))


    def evaluate(model, test_features, test_labels):
        predictions = model.predict(test_features)

        result = model.predict_proba(test_features) # 첫 클래스의 예측확률, 두번째 클래스의 예측 확률 어떤것이 0, 1이지?
        y_pred = model.predict(test_features)

        conf_mat = confusion_matrix(test_labels,y_pred,labels=[0,1])
        tn, fp, fn, tp = conf_mat.ravel()

        #auc_roc
        try:
            auc_roc =  roc_auc_score(test_labels,result[:,1] )
        except ValueError:
            auc_roc = 0

        #auc_pr
        try : 
            precision_l, recall_l, thresholds = precision_recall_curve(test_labels,result[:,1])
            auc_pr = auc(recall_l, precision_l)
            if np.isnan(auc_pr) : 
                auc_pr = 0
        except : 
            auc_pr = 0


        #recall
        if (tp+fn) == 0 :
            recall = 0
        else : 
            recall = tp/(tp+fn)

        if np.isnan(recall) :
            recall = 0

        #precision
        if (tp+fp) == 0 :
            precision = 0
        else : 
            precision = tp/(tp+fp)

        if np.isnan(precision) :
            precision = 0

        #F1
        if recall == 0 or precision ==0  : 
            F1 = 0
        else : 
            F1 = 2* (precision * recall) / (precision + recall)

        if np.isnan(F1) : 
            F1 = 0 

        #
        specificity = tn / (fp+tn) # 실제 0인 것중 모델이 0이라한것의 비율

        b_a = (recall + specificity) / 2


        return auc_roc, auc_pr, F1, b_a, precision, recall, specificity, tp, tn, fp, fn

    def Average(lst):
        return sum(lst) / len(lst)

    #dataframe to record performance
    result_df = pd.DataFrame(columns = ['Inside','AUC_ROC', 'AUC_PR', 'F1', 'B_A', 'Precision', 'Recall', 'Specificity', 'TP', 'TN', 'FP', 'FN',
                                                        'Outside','AUC_ROC_o', 'AUC_PR_o', 'F1_o', 'B_A_o', 
                                        'Precision_o', 'Recall_o', 'Specificity_o', 'TP_o', 'TN_o', 'FP_o', 'FN_o']) 

    params_rf = dict(BestParam.loc[assay_index][1:])
    if params_rf['max_depth'] != 'None' : 
        params_rf['max_depth'] = int(params_rf['max_depth'])
    else : 
        params_rf['max_depth'] = None
    
    X_train_valid, y_train_valid = sme.fit_sample(X_train_valid, y_train_valid )

    #test_tuned
    best_random = RandomForestClassifier(random_state = 42).set_params(**params_rf)
    best_random.fit(X_train_valid, y_train_valid)


    auc_roc, auc_pr, F1, b_a, precision, recall, specificity, tp, tn, fp, fn = evaluate(best_random, X_test[inside_AD], y_test[inside_AD])

    auc_roc_o, auc_pr_o, F1_o, b_a_o, precision_o, recall_o, specificity_o, tp_o, tn_o, fp_o, fn_o = evaluate(best_random, X_test[outside_AD], y_test[outside_AD])

    result_df = result_df.append(pd.DataFrame([[len(inside_AD),auc_roc, auc_pr, F1, b_a, precision, recall, specificity, tp, tn, fp, fn,
                                                len(outside_AD),auc_roc_o, auc_pr_o, F1_o, b_a_o, precision_o, recall_o, specificity_o, tp_o, tn_o, fp_o, fn_o]], 
                                              columns = ['Inside','AUC_ROC', 'AUC_PR', 'F1', 'B_A', 'Precision', 'Recall', 'Specificity', 'TP', 'TN', 'FP', 'FN',
                                                        'Outside','AUC_ROC_o', 'AUC_PR_o', 'F1_o', 'B_A_o', 'Precision_o', 'Recall_o', 'Specificity_o', 'TP_o', 'TN_o', 'FP_o', 'FN_o']) )

    # error.. ZeroDivisionError: division by zero
    #     print('Improvement of {:0.2f}%.'.format( 100 * (auc_t - auc) / auc))
    #     print('Improvement of {:0.2f}%.'.format( 100 * (recall_t - recall) / recall))
    #     print('Improvement of {:0.2f}%.'.format( 100 * (F1_t - F1) / F1))

    result_df.index = [ss]
    result_df.to_csv(save_path+'performance/performance'+ss+".csv")

In [31]:
# NVS_ADME_hCYP1A2
ss = 'NVS_ADME_hCYP1A2' # 그룹
ffpp = "maccs"

df_ss = df.loc[:,['smiles',ss]]
df_ss = df_ss.dropna()
assays_ss = df_ss.columns
sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

    #Smiles --> MACCS Keys
assay = ss
df_train  = df_ss[['smiles',ss] ]

smiles_list = []
err_smiles = []# fingerfrint로 변환시 에러 smiles
fps = []
enc_y = []
for index, row in df_train.iterrows():

  if row[-1] == 'N' : 
    target = 0
  else :
    target = 1

  try : 
    mol = Chem.MolFromSmiles(row['smiles'])

    if ffpp == 'maccs' :    
        fp = MACCSkeys.GenMACCSKeys(mol)
    elif ffpp == 'morgan' : 
        fp = Chem.AllChem.GetMorganFingerprintAsBitVect(mol, 2)
    elif ffpp == 'rdkit' : 
        fp = Chem.RDKFingerprint(mol)
    elif ffpp == 'pattern' : 
        fp = Chem.rdmolops.PatternFingerprint(mol)
    elif ffpp == 'layerd' : 
        fp = Chem.rdmolops.LayeredFingerprint(mol)
    else : 
        print("fingerfrint selection error!")

    smiles_list.append(row['smiles'])   
    fps.append(fp)
    enc_y.append(target)
  except : 
    err_smiles.append(row['smiles'])
    pass

np_fps = []
for fp in fps:
  arr = numpy.zeros((1,))
  DataStructs.ConvertToNumpyArray(fp, arr)
  np_fps.append(arr)

np_fps_array = numpy.array(np_fps)

print("변환 에러난 smiles 개수 : ", len(err_smiles), err_smiles)

x_train_res, y_train_res = np_fps_array, np.array(enc_y)
train = pd.DataFrame(data=x_train_res)
train['label'] = y_train_res

d = pd.DataFrame(columns = ['smiles','Type'] )
d['smiles'] = smiles_list
d['Type'] = enc_y

#모든 물질에 대한 fingerprint를 string형태로 바꾼다(비교하기위함).
mychems = []
for item in np_fps:
    #print(item)
    item = np.array(item,dtype=int) #왜 굳이 두번? int형으로 추가하려면 이렇게 하면 되는구나....
    s = item.tolist() #넘파이어레이를 리스트로 바꾼다. 
    t = [str(i) for i in s]
    mychems.append(''.join(t))

i = 0
counter = 0
chemdict = {} #딗셔너리.
listofdups = []
for item in mychems:
    try:
        chemdict[str(item)].append(i)
        #print('dup found')
        for j in range(len(chemdict[str(item)])):
            if (d['Type'][i] != d['Type'][chemdict[str(item)][j]]) :
                listofdups.append(i)
                counter+=1
    except: 
        #print('adding new')
        chemdict[str(item)] = []
        chemdict[str(item)].append(i)
    i+=1
print(str(counter)+' '+str(len(list(set(listofdups)))))
#smote적용하기 전에서 listofdups를 빼면 된다!

dups_list = list(set(listofdups))
pprint(dups_list) # 0인것만 지우는게 맞나?

dups_list_0 = []
for index in dups_list : 
  if train.iloc[index].label == 0 :
    dups_list_0.append(index)

train = train.drop(dups_list_0)

duplicated_smiles_0_df = pd.DataFrame()
duplicated_smiles_0_df['deleted_duplicate_smiles_0'] = smiles_list
duplicated_smiles_0_df = duplicated_smiles_0_df.iloc[dups_list_0,]
duplicated_smiles_0_df

save_path = './31assays_AD/'
dupdf = save_path+"dup_smiles/"+ss+"_dup_smiles_df.csv"
duplicated_smiles_0_df.to_csv(dupdf)

err_smiles_df = pd.DataFrame(columns = [ss], index = ['err_n','err_smiles'])
err_smiles_df[ss] = [ len(err_smiles), err_smiles]

ersdf = save_path+"err_smiles/"+ss+"_err_smiles_df.csv"
err_smiles_df.to_csv(ersdf)


skf = StratifiedKFold(n_splits=10)
sme = SMOTEENN(random_state=42,sampling_strategy = "minority")

X = np.array( train.drop('label', axis =1) )
y = np.array( train.label )

#train test split
X_train_valid, X_test, y_train_valid, y_test = train_test_split(X, y,
                                                    stratify=y, 
                                                    test_size=0.1,
                                                    random_state = 42)


sklearn_pca = sklearnPCA(n_components=2)
y_sklearn = sklearn_pca.fit_transform(X_train_valid)
pccr = pandas.DataFrame(data=y_sklearn)
pccr.columns = ['PC1', 'PC2']

y_sklearn2 = sklearn_pca.fit_transform(X_test)
pccr2 = pandas.DataFrame(data=y_sklearn2)
pccr2.columns = ['PC1', 'PC2']

def is_p_inside_points_hull(points, p):
    """Check if a point is inside the convex hull."""
    hull = ConvexHull(points)
    new_points = numpy.append(points, p, axis=0)
    new_hull = ConvexHull(new_points)
    if list(hull.vertices) == list(new_hull.vertices):
        return True
    else:
        return False

inside_AD = []
outside_AD = []
for i in range(0, len(pccr2)):
    if is_p_inside_points_hull(pccr, pccr2.loc[[i]]) : 
        inside_AD.append(i)
    else : 
        outside_AD.append(i)

print('inside_AD : ', len(inside_AD))
print('outside_AD : ', len(outside_AD))


def evaluate(model, test_features, test_labels):
    predictions = model.predict(test_features)

    result = model.predict_proba(test_features) # 첫 클래스의 예측확률, 두번째 클래스의 예측 확률 어떤것이 0, 1이지?
    y_pred = model.predict(test_features)

    conf_mat = confusion_matrix(test_labels,y_pred,labels=[0,1])
    tn, fp, fn, tp = conf_mat.ravel()

    #auc_roc
    try:
        auc_roc =  roc_auc_score(test_labels,result[:,1] )
    except ValueError:
        auc_roc = 0

    #auc_pr
    try : 
        precision_l, recall_l, thresholds = precision_recall_curve(test_labels,result[:,1])
        auc_pr = auc(recall_l, precision_l)
        if np.isnan(auc_pr) : 
            auc_pr = 0
    except : 
        auc_pr = 0


    #recall
    if (tp+fn) == 0 :
        recall = 0
    else : 
        recall = tp/(tp+fn)

    if np.isnan(recall) :
        recall = 0

    #precision
    if (tp+fp) == 0 :
        precision = 0
    else : 
        precision = tp/(tp+fp)

    if np.isnan(precision) :
        precision = 0

    #F1
    if recall == 0 or precision ==0  : 
        F1 = 0
    else : 
        F1 = 2* (precision * recall) / (precision + recall)

    if np.isnan(F1) : 
        F1 = 0 

    #
    specificity = tn / (fp+tn) # 실제 0인 것중 모델이 0이라한것의 비율

    b_a = (recall + specificity) / 2


    return auc_roc, auc_pr, F1, b_a, precision, recall, specificity, tp, tn, fp, fn

def Average(lst):
    return sum(lst) / len(lst)

#dataframe to record performance
result_df = pd.DataFrame(columns = ['Inside','AUC_ROC', 'AUC_PR', 'F1', 'B_A', 'Precision', 'Recall', 'Specificity', 'TP', 'TN', 'FP', 'FN',
                                                    'Outside','AUC_ROC_o', 'AUC_PR_o', 'F1_o', 'B_A_o', 
                                    'Precision_o', 'Recall_o', 'Specificity_o', 'TP_o', 'TN_o', 'FP_o', 'FN_o']) 

params_rf = dict(BestParam.loc[assay_index][1:])
if params_rf['max_depth'] != 'None' : 
    params_rf['max_depth'] = int(params_rf['max_depth'])
else : 
    params_rf['max_depth'] = None

X_train_valid, y_train_valid = sme.fit_sample(X_train_valid, y_train_valid )

#test_tuned
best_random = RandomForestClassifier(random_state = 42).set_params(**params_rf)
best_random.fit(X_train_valid, y_train_valid)


auc_roc, auc_pr, F1, b_a, precision, recall, specificity, tp, tn, fp, fn = evaluate(best_random, X_test[inside_AD], y_test[inside_AD])

auc_roc_o, auc_pr_o, F1_o, b_a_o, precision_o, recall_o, specificity_o, tp_o, tn_o, fp_o, fn_o = evaluate(best_random, X_test[outside_AD], y_test[outside_AD])

result_df = result_df.append(pd.DataFrame([[len(inside_AD),auc_roc, auc_pr, F1, b_a, precision, recall, specificity, tp, tn, fp, fn,
                                            len(outside_AD),auc_roc_o, auc_pr_o, F1_o, b_a_o, precision_o, recall_o, specificity_o, tp_o, tn_o, fp_o, fn_o]], 
                                          columns = ['Inside','AUC_ROC', 'AUC_PR', 'F1', 'B_A', 'Precision', 'Recall', 'Specificity', 'TP', 'TN', 'FP', 'FN',
                                                    'Outside','AUC_ROC_o', 'AUC_PR_o', 'F1_o', 'B_A_o', 'Precision_o', 'Recall_o', 'Specificity_o', 'TP_o', 'TN_o', 'FP_o', 'FN_o']) )

# error.. ZeroDivisionError: division by zero
#     print('Improvement of {:0.2f}%.'.format( 100 * (auc_t - auc) / auc))
#     print('Improvement of {:0.2f}%.'.format( 100 * (recall_t - recall) / recall))
#     print('Improvement of {:0.2f}%.'.format( 100 * (F1_t - F1) / F1))

result_df.index = [ss]
result_df.to_csv(save_path+'performance/performance'+ss+".csv")

변환 에러난 smiles 개수 :  0 []
1 1
[65]
inside_AD :  16
outside_AD :  2


/tmp/ipykernel_6729/721203879.py:218: RuntimeWarning: invalid value encountered in long_scalars
  specificity = tn / (fp+tn) # 실제 0인 것중 모델이 0이라한것의 비율


In [22]:
assays_index_list = [i for i in range(31)]

In [23]:
from rdkit import RDLogger                                                                                                                                                               
RDLogger.DisableLog('rdApp.*') 

In [24]:
import math 
import time 
import datetime 
start = time.time() 

In [25]:
import multiprocessing
CPU_CORE = multiprocessing.cpu_count() # 멀티프로세싱 CPU 사용 수
if __name__=='__main__':
    pool = multiprocessing.Pool(processes=CPU_CORE)
    pool.map(g_tf, assays_index_list)
    pool.close()
    pool.join()  

변환 에러난 smiles 개수 :  0 []
0 0
[]
변환 에러난 smiles 개수 :  0 []
inside_AD : 1 1 
[65]
8
outside_AD :  6
inside_AD :  16
outside_AD :  2


/tmp/ipykernel_6729/4021596425.py:218: RuntimeWarning: invalid value encountered in long_scalars
  specificity = tn / (fp+tn) # 실제 0인 것중 모델이 0이라한것의 비율


변환 에러난 smiles 개수 :  2 ['CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1']
27 18
[193,
 354,
 545,
 196,
 388,
 358,
 383,
 516,
 546,
 508,
 464,
 189,
 505,
 378,
 347,
 316,
 221,
 351]
inside_AD :  49
outside_AD :  7


/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]


변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])(

/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/pytho

In [26]:
end = time.time() 
sec = (end - start) 
result = datetime.timedelta(seconds=sec) 
print(result) 
result_list = str(datetime.timedelta(seconds=sec)).split(".") 
print(result_list[0])

#우선.. 8시간 17분 소요

0:01:16.268417
0:01:16


In [27]:
import os
finished_list = os.listdir("./31assays_AD/performance")

In [28]:
finished_assays=[]
for i in finished_list : 
    i2 = i.replace("performance","").replace(".csv","")
    finished_assays.append(i2)

In [29]:
not_finished_assays = list(set(selected_assay_list).difference(set(finished_assays)))

In [30]:
not_finished_assays

[]

In [30]:
not_finished_index =[]
for i in not_finished_assays : 
    not_finished_index.append( selected_assay_list.index(i) )

In [24]:
not_finished_index

[9, 25, 29, 26, 11, 5, 0, 3, 13, 17, 27]

In [38]:
import multiprocessing
CPU_CORE = multiprocessing.cpu_count() # 멀티프로세싱 CPU 사용 수
if __name__=='__main__':
    pool = multiprocessing.Pool(processes=CPU_CORE)
    pool.map(g_tf, not_finished_index)
    pool.close()
    pool.join()  

변환 에러난 smiles 개수 :  0 []
1 1
[65]
변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-

/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/python3.9/site-packages/sklearn/metrics/_ranking.py:817: RuntimeWarning: invalid value encountered in true_divide
  recall = tps / tps[-1]
/home2/prof/chemdsrc/anaconda3/envs/molpmofit/lib/pytho

In [25]:
!pwd

/home2/prof/chemdsrc/UOSEST/CI-DS-ToxCast-ML-main


In [76]:
selected_assay_list.index('TOX21_VDR_BLA_agonist_ratio')

5

In [ ]:
for i in not_finished_assays : 
    g_tf(selected_assay_list.index(i))

In [37]:
g_tf(selected_assay_list.index('NVS_ADME_hCYP1A2'))

변환 에러난 smiles 개수 :  0 []
1 1
[65]


In [16]:
g_tf(selected_assay_list.index('TOX21_PPARd_BLA_antagonist_ratio'))

변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
29 24
[5891,
 3847,
 524,
 3471,
 3472,
 6814,
 4001,
 3618,
 5028,
 2601,
 2602,
 2603,
 6192,
 3275,
 3276,
 6859,
 6098,
 2775,
 6743,
 5980,
 5992,
 3821,
 3702,
 3703]
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] END rf__bootstrap=False, rf__max_depth=10, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=1200; total time=  35.8s
[CV] END rf__bootstrap=False, rf__max_depth=60, rf__max_features=s

In [22]:
g_tf(selected_assay_list.index('TOX21_VDR_BLA_agonist_ratio'))

변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
8 7
[6816, 4611, 3082, 2871, 5370, 6875, 6909]
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


In [23]:
g_tf(selected_assay_list.index('TOX21_ELG1_LUC_Agonist'))

변환 에러난 smiles 개수 :  12 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[H][O]1[Au+][S-]C2O[C@H](CO)[C@@H](O)[C@H](O)[C@@]12[H]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[NH4+].[NH4+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
7 7
[5666, 7557, 5575, 6631, 4807, 6764, 6644]
[CV] END rf__bootstrap=True, rf__max_depth=10, rf__max_features=sqrt, rf__min_samples_leaf=4, rf__min_samples_split=2, rf__n_estimators=1600; total time=  35.4s
[CV] END rf__bootstrap=False, rf__max_depth=100, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=1000; total time=  40.5

In [24]:
g_tf(selected_assay_list.index('TOX21_TR_LUC_GH3_Agonist'))

변환 에러난 smiles 개수 :  12 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[H][O]1[Au+][S-]C2O[C@H](CO)[C@@H](O)[C@H](O)[C@@]12[H]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[NH4+].[NH4+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
2 1
[1614]
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] END rf__bootstrap=True, rf__max_depth=10, rf__max_features=sqrt, rf__min_samples_leaf=4, rf__min_samples_split=2, rf__n_estimators=1600; total time=  35.9s
[CV] END rf__bootstrap=False, rf__max_depth=100, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estim

In [25]:
g_tf(selected_assay_list.index('TOX21_ESRE_BLA_ratio'))

변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
38 23
[2560,
 4500,
 2590,
 6816,
 6817,
 5028,
 4389,
 2355,
 1331,
 2871,
 6585,
 4795,
 2371,
 6860,
 6098,
 6738,
 6875,
 5980,
 6876,
 4837,
 4198,
 5992,
 5370]
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] END rf__bootstrap=False, rf__max_depth=30, rf__max_features=auto, rf__min_samples_leaf=4, rf__min_samples_split=2, rf__n_estimators=2000; total time= 1.1min
[CV] END rf__bootstrap=True, rf__max_depth=70, rf__max_features=auto, rf

In [26]:
g_tf(selected_assay_list.index('TOX21_FXR_BLA_agonist_ratio'))

변환 에러난 smiles 개수 :  10 ['[F-][B+3]([F-])([F-])[F-].CC[N+]1(C)CCCC1', 'CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1', '[NH4+].[NH4+].[Cl-][Pt++]([Cl-])([Cl-])[Cl-]', '[Cl-][Pt]1([Cl-])[NH2+]CC[NH2+]1', '[H][N]([H])([H])[Pt++]([Cl-])([Cl-])[N]([H])([H])[H]', '[Na+].[Na+].F[Si--](F)(F)(F)(F)F', '[Cl-][Fe+3]([Cl-])([Cl-])[Cl-].CCCCN1C=C[N+](C)=C1', '[H][N]([H])([H])[Pt++]1([O-]C(=O)C2(CCC2)C(=O)[O-]1)[N]([H])([H])[H]', 'C[Sn](C)(|[O]C(=O)CCCCCC(C)(C)C)|[O]C(=O)CCCCCC(C)(C)C']
10 8
[6816, 2497, 4837, 4389, 6643, 2871, 5370, 6875]
Fitting 10 folds for each of 100 candidates, totalling 1000 fits
[CV] END rf__bootstrap=True, rf__max_depth=10, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=2000; total time=  42.2s
[CV] END rf__bootstrap=False, rf__max_depth=60, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=600; total time=  26.5s
[CV] END rf__bootstrap=Tr

In [27]:
end = time.time() 
sec = (end - start) 
result = datetime.timedelta(seconds=sec) 
print(result) 
result_list = str(datetime.timedelta(seconds=sec)).split(".") 
print(result_list[0])

#우선.. 8시간 17분 소요

1:46:35.680049
1:46:35


In [ ]:
os.listdir('')

In [24]:
# for i in assays_index_list :
#     g_tf(i)

변환 에러난 smiles 개수 :  2 ['CC1C=C[C-](=C1)[Mn+]([C-]#[O+])([C-]#[O+])[C-]#[O+]', '[O-]1N2C=CC=CC2=[S][Zn++]11[O-]N2C=CC=CC2=[S]1']
27 18
[193,
 354,
 545,
 196,
 388,
 358,
 383,
 516,
 546,
 508,
 464,
 189,
 505,
 378,
 347,
 316,
 221,
 351]
Model Performance
AUC = 0.80%.
Recall = 0.61%.
F1 = 0.68%.
Model Performance
AUC = 0.79%.
Recall = 0.76%.
F1 = 0.73%.
Model Performance
AUC = 0.78%.
Recall = 0.54%.
F1 = 0.65%.
Model Performance
AUC = 0.86%.
Recall = 0.76%.
F1 = 0.78%.
Model Performance
AUC = 0.82%.
Recall = 0.78%.
F1 = 0.68%.
Model Performance
AUC = 0.70%.
Recall = 0.56%.
F1 = 0.50%.
Model Performance
AUC = 0.76%.
Recall = 0.62%.
F1 = 0.67%.
Model Performance
AUC = 0.76%.
Recall = 0.74%.
F1 = 0.68%.
Model Performance
AUC = 0.83%.
Recall = 0.79%.
F1 = 0.71%.
Model Performance
AUC = 0.83%.
Recall = 0.59%.
F1 = 0.72%.
Fitting 10 folds for each of 100 candidates, totalling 1000 fits


Process LokyProcess-557:
Process LokyProcess-558:
Process LokyProcess-560:
Process LokyProcess-561:
Process LokyProcess-553:
Process LokyProcess-559:
Process LokyProcess-555:
Process LokyProcess-550:
Process LokyProcess-547:
Process LokyProcess-545:
Process LokyProcess-554:
Process LokyProcess-556:
Process LokyProcess-548:
Process LokyProcess-551:
Process LokyProcess-549:
Process LokyProcess-542:
Process LokyProcess-552:
Process LokyProcess-544:
Process LokyProcess-543:
Process LokyProcess-546:
Process LokyProcess-541:
Process LokyProcess-532:
Process LokyProcess-535:
Process LokyProcess-530:
Process LokyProcess-540:
Process LokyProcess-531:
Process LokyProcess-539:
Process LokyProcess-536:
Process LokyProcess-538:
Process LokyProcess-537:
Process LokyProcess-534:
Process LokyProcess-527:
Process LokyProcess-526:
Process LokyProcess-533:
Process LokyProcess-523:
Process LokyProcess-525:
Process LokyProcess-522:
Process LokyProcess-529:
Process LokyProcess-528:
Process LokyProcess-520:


[CV] END rf__bootstrap=True, rf__max_depth=10, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=2000; total time=   2.9s
[CV] END rf__bootstrap=False, rf__max_depth=10, rf__max_features=sqrt, rf__min_samples_leaf=2, rf__min_samples_split=5, rf__n_estimators=1600; total time=   1.6s
[CV] END rf__bootstrap=False, rf__max_depth=110, rf__max_features=auto, rf__min_samples_leaf=2, rf__min_samples_split=10, rf__n_estimators=1800; total time=   1.8s
[CV] END rf__bootstrap=False, rf__max_depth=80, rf__max_features=sqrt, rf__min_samples_leaf=1, rf__min_samples_split=5, rf__n_estimators=1400; total time=   1.5s
[CV] END rf__bootstrap=False, rf__max_depth=20, rf__max_features=sqrt, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=1200; total time=   1.2s
[CV] END rf__bootstrap=True, rf__max_depth=50, rf__max_features=sqrt, rf__min_samples_leaf=4, rf__min_samples_split=10, rf__n_estimators=800; total time=   1.0s
[CV] END rf__bootstrap=True

KeyboardInterrupt: 